In [ ]:
import pandas as pd
import numpy as np

## Get the data

In [ ]:
metadata = pd.read_csv('movies_metadata.csv')

In [ ]:
metadata.head()

In [ ]:
metadata.columns

## Clean the data
As for now I am only focusing on genres, title, overview and tagline

In [ ]:
metadata = metadata.drop_duplicates(subset=['id'])

In [ ]:
movie_dt = metadata[['genres', 'title', 'overview', 'tagline']]

In [ ]:
movie_dt.head()

In [ ]:
movie_dt.isna().sum()

In [ ]:
movie_dt.shape

Cleaning genre column and keeping list of genres for each movie

In [ ]:
movie_dt.genres[0]

In [ ]:
from ast import literal_eval
movie_dt.genres = movie_dt.genres.fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
movie_dt.genres[0]

In [ ]:
movie_dt.head()

## Create feature vactors
Combine `overview` and `tagline` together and name the column as `content`. <br/> Convert `genres` column from list to continuous string. <br/> Then apply TFIDF on `content` column and CountVectorizer on `genres` column.

In [ ]:
movie_dt.tagline = movie_dt.tagline.fillna('')
movie_dt.overview = movie_dt.overview.fillna('')
movie_dt['content'] = movie_dt.tagline + movie_dt.overview

In [ ]:
movie_dt.head()

In [ ]:
final_df = movie_dt[['genres', 'title', 'content']]
final_df['genres'] = final_df['genres'].apply(lambda x : ' '.join(x))
final_df.dropna(inplace=True)
final_df.head()

In [ ]:
pd.concat(g for _, g in final_df.groupby("title") if len(g) > 1)

There are some duplicates. For the sake of this task, I am assuming all the duplicates actually indicates the same movie but different descriptions. So I am dropping the duplicate titles.

In [ ]:
final_df = final_df.drop_duplicates(subset=['title'])
final_df.head()

## Create the recommendation system
Give 50-50 weightage for both genre tags and content

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_content = tf.fit_transform(final_df['content'])

In [ ]:
cosine_sim_content = linear_kernel(tfidf_content, tfidf_content)

In [ ]:
np.save('content.npy', cosine_sim_content)

In [ ]:
count_vectorizer = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_genre = count_vectorizer.fit_transform(final_df['genres'])

In [ ]:
cosine_sim_genre = cosine_similarity(count_genre, count_genre)

In [ ]:
np.save('genre.npy', cosine_sim_genre)

In [ ]:
final_df = final_df.reset_index()
titles = final_df['title']
indices = pd.Series(final_df.index, index=final_df['title'])

In [ ]:
indices.to_pickle('movie_titles.pkl')

In [ ]:
# content = np.load('content.npy')
# genre = np.load('genre.npy')
movie_titles = pd.read_pickle('movie_titles.pkl')

In [ ]:
def get_recommendations(title):
    idx = movie_titles[title]
    sim_scores = list(enumerate(content[idx])) + list(enumerate(genre[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
get_recommendations('Interstellar')

In [ ]:
get_recommendations('The Dark Knight')

## convert